<a href="https://colab.research.google.com/github/edquestofficial/Gen-AI-Cohort/blob/main/2024/april/Level_2/LLaMA_Index/LLaMA_Index_JSON_Query_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JSON Query Engine

## Reference
* [YouTube](https://www.youtube.com/watch?v=4tDyfAaIqEw&list=PLTZkGHtR085ZjK1srrSZIrkeEzQiMjO9W&index=5)
* [Documentation](https://docs.llamaindex.ai/en/stable/examples/query_engine/json_query_engine/)

In [ ]:
! pip install llama-index

In [ ]:
! pip install -q llama-index-llms-gemini

In [ ]:
# First, install the jsonpath-ng package which is used by default to parse & execute the JSONPath queries.
! pip install jsonpath-ng

In [4]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Mount Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
import os

base_path = "/content/drive/MyDrive/Gen AI Course/RAG_For_HDFC_Policy"
filepath = f"{base_path}/gemini_api_key.txt"
with open(filepath, "r") as f:
  api_key = ' '.join(f.readlines())
  os.environ["GOOGLE_API_KEY"] = api_key

In [7]:
from IPython.display import Markdown, display

In [8]:
from llama_index.llms.gemini import Gemini

llm = Gemini(api_key=os.environ["GOOGLE_API_KEY"], model_name="models/gemini-pro")

## Let's start on a Toy JSON

Very simple JSON object containing data from a blog post site with user comments.

We will also provide a JSON schema (which we were able to generate by giving ChatGPT a sample of the JSON).

### Advice
Do make sure that you've provided a helpful `"description"` value for each of the fields in your JSON schema.

As you can see in the given example, the description for the `"username"` field mentions that usernames are lowercased. You'll see that this ends up being helpful for the LLM in producing the correct JSON path query.

In [9]:
# Test on some sample data
json_value = {
    "blogPosts": [
        {
            "id": 1,
            "title": "First blog post",
            "content": "This is my first blog post",
        },
        {
            "id": 2,
            "title": "Second blog post",
            "content": "This is my second blog post",
        },
    ],
    "comments": [
        {
            "id": 1,
            "content": "Nice post!",
            "username": "jerry",
            "blogPostId": 1,
        },
        {
            "id": 2,
            "content": "Interesting thoughts",
            "username": "simon",
            "blogPostId": 2,
        },
        {
            "id": 3,
            "content": "Loved reading this!",
            "username": "simon",
            "blogPostId": 2,
        },
    ],
}

In [19]:
# JSON Schema object that the above JSON value conforms to
json_schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "description": "Schema for a very simple blog post app",
    "type": "object",
    "properties": {
        "blogPosts": {
            "description": "List of blog posts",
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "id": {
                        "description": "Unique identifier for the blog post",
                        "type": "integer",
                    },
                    "title": {
                        "description": "Title of the blog post",
                        "type": "string",
                    },
                    "content": {
                        "description": "Content of the blog post",
                        "type": "string",
                    },
                },
                "required": ["id", "title", "content"],
            },
        },
        "comments": {
            "description": "List of comments on blog posts",
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "id": {
                        "description": "Unique identifier for the comment",
                        "type": "integer",
                    },
                    "content": {
                        "description": "Content of the comment",
                        "type": "string",
                    },
                    "username": {
                        "description": (
                            "Username of the commenter (lowercased)"
                        ),
                        "type": "string",
                    },
                    "blogPostId": {
                        "description": (
                            "Identifier for the blog post to which the comment"
                            " belongs"
                        ),
                        "type": "integer",
                    },
                },
                "required": ["id", "content", "username", "blogPostId"],
            },
        },
    },
    "required": ["blogPosts", "comments"],
}

In [20]:
from llama_index.core.indices.struct_store import JSONQueryEngine

nl_query_engine = JSONQueryEngine(
    json_value=json_value,
    json_schema=json_schema,
    llm=llm,
)
raw_query_engine = JSONQueryEngine(
    json_value=json_value,
    json_schema=json_schema,
    llm=llm,
    synthesize_response=False,
)

## Test

In [25]:
nl_response = nl_query_engine.query(
    "What comments has simon been writing?",
)
raw_response = raw_query_engine.query(
    "What comments has simon been writing?",
)

In [26]:
display(
    Markdown(f"<h1>Natural language Response</h1><br><b>{nl_response}</b>")
)
display(Markdown(f"<h1>Raw JSON Response</h1><br><b>{raw_response}</b>"))

<h1>Natural language Response</h1><br><b>simon has written the following comments: "Interesting thoughts" and "Loved reading this!".</b>

<h1>Raw JSON Response</h1><br><b>{"username==\"simon\")]": "{'id': 2, 'content': 'Interesting thoughts', 'username': 'simon', 'blogPostId': 2}, {'id': 3, 'content': 'Loved reading this!', 'username': 'simon', 'blogPostId': 2}"}</b>

In [27]:
# get the json path query string. Same would apply to raw_response
print(nl_response.metadata["json_path_response_str"])

$.comments[?(@.username=="simon")]


## Another Test

In [23]:
nl_response = nl_query_engine.query(
    "What comments has Jerry been writing?",
)
raw_response = raw_query_engine.query(
    "What comments has Jerry been writing?",
)

In [24]:
display(
    Markdown(f"<h1>Natural language Response</h1><br><b>{nl_response}</b>")
)
display(Markdown(f"<h1>Raw JSON Response</h1><br><b>{raw_response}</b>"))

<h1>Natural language Response</h1><br><b>I cannot answer this question because there are no comments by Jerry in the provided JSON.</b>

<h1>Raw JSON Response</h1><br><b>{"username == \"jerry\")]": "{'id': 1, 'content': 'Nice post!', 'username': 'jerry', 'blogPostId': 1}"}</b>

In [22]:
# get the json path query string. Same would apply to raw_response
print(nl_response.metadata["json_path_response_str"])

$.comments[?(@.username=="Jerry")]
